# This Notebook is for the transformation of a pre-trained LPYOLO model
# using FINN
## It is the same as the one for the ZCU102. However, the parallelism values were reduced to allow for deploying the porject on PYNQ-board since it has more limited resources

In [1]:
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
import os
import onnx
import torch 

build_dir = '/home/drmervat/Desktop/YOLO_GP/YOLO__'

In [2]:
import torch
import onnx
from finn.util.test import get_test_model_trained
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

In [3]:
export_onnx_path = build_dir + "/yolo_export.onnx"

In [4]:
showInNetron(build_dir+"/yolo_export.onnx")

Serving '/home/drmervat/Desktop/YOLO_GP/YOLO__/yolo_export.onnx' at http://0.0.0.0:8081


In [5]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

model = ModelWrapper(export_onnx_path)
model = model.transform(ConvertQONNXtoFINN())
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())
model.save(build_dir + "/yolo_tidy.onnx")

In [6]:
showInNetron(build_dir+"/yolo_tidy.onnx")

Stopping http://0.0.0.0:8081
Serving '/home/drmervat/Desktop/YOLO_GP/YOLO__/yolo_tidy.onnx' at http://0.0.0.0:8081


In [7]:
from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup


model = ModelWrapper(build_dir+"/yolo_tidy.onnx")
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = build_dir+"/yolo_preproc.onnx"
export_qonnx(totensor_pyt, torch.randn(ishape), chkpt_preproc_name)
qonnx_cleanup(chkpt_preproc_name, out_file=chkpt_preproc_name)
pre_model = ModelWrapper(chkpt_preproc_name)
pre_model = pre_model.transform(ConvertQONNXtoFINN())

# join preprocessing and core model
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all BNN-PYNQ models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])

/home/drmervat/Desktop/finn/deps/qonnx/src/qonnx/transformation/merge_onnx_models.py:70: UserWarning: [MergeONNXModels] opsets for models to merge differ: 14 vs 17, output model will use opset 17
  warnings.warn(
/home/drmervat/Desktop/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


In [8]:
from qonnx.transformation.insert_topk import InsertTopK
from qonnx.transformation.infer_datatypes import InferDataTypes
chkpt_name = build_dir+"/yolo_preproc_tidy.onnx"
# tidy-up again
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model.save(chkpt_name)

In [9]:
showInNetron(build_dir+"/yolo_preproc_tidy.onnx")

Stopping http://0.0.0.0:8081
Serving '/home/drmervat/Desktop/YOLO_GP/YOLO__/yolo_preproc_tidy.onnx' at http://0.0.0.0:8081


In [10]:
from finn.transformation.streamline import Streamline
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveScalarLinearPastInvariants
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.general import RemoveUnusedTensors


In [11]:
model = ModelWrapper(build_dir + "/yolo_preproc_tidy.onnx")

In [12]:
model = model.transform(MoveScalarLinearPastInvariants())
model = model.transform(Streamline())
model = model.transform(LowerConvsToMatMul())
model = model.transform(MakeMaxPoolNHWC())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(ConvertBipolarMatMulToXnorPopcount())
model = model.transform(Streamline())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())
model.save(build_dir + "/yolo_streamlined.onnx")


In [13]:
showInNetron(build_dir+"/yolo_streamlined.onnx")

Stopping http://0.0.0.0:8081
Serving '/home/drmervat/Desktop/YOLO_GP/YOLO__/yolo_streamlined.onnx' at http://0.0.0.0:8081


In [14]:
from finn.util.basic import pynq_part_map
# change this if you have a different PYNQ board, see list above
pynq_board = "Pynq-Z2"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

In [15]:
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers
from qonnx.custom_op.registry import getCustomOp
from qonnx.transformation.infer_data_layouts import InferDataLayouts


In [16]:
model = ModelWrapper(build_dir + "/yolo_streamlined.onnx")
model = model.transform(to_hw.InferBinaryMatrixVectorActivation())
model = model.transform(to_hw.InferQuantizedMatrixVectorActivation())
model = model.transform(to_hw.InferThresholdingLayer())
model = model.transform(to_hw.InferConvInpGen())
model = model.transform(to_hw.InferStreamingMaxPool())
model = model.transform(absorb.AbsorbConsecutiveTransposes())
model = model.transform(RemoveCNVtoFCFlatten())
parent_model = model.transform(CreateDataflowPartition())
parent_model.save(build_dir + "/yolo_dataflow_parent.onnx")
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_model_filename = sdp_node.get_nodeattr("model")
# save the dataflow partition with a different name for easier access
# and specialize the layers to HLS variants
dataflow_model = ModelWrapper(dataflow_model_filename)
dataflow_model = dataflow_model.transform(SpecializeLayers(fpga_part))
dataflow_model.save(build_dir + "/yolo_dataflow_model.onnx")

In [17]:
showInNetron(build_dir + "/yolo_dataflow_parent.onnx")

Stopping http://0.0.0.0:8081
Serving '/home/drmervat/Desktop/YOLO_GP/YOLO__/yolo_dataflow_parent.onnx' at http://0.0.0.0:8081


In [18]:
showInNetron(build_dir + "/yolo_dataflow_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/home/drmervat/Desktop/YOLO_GP/YOLO__/yolo_dataflow_model.onnx' at http://0.0.0.0:8081


In [19]:
model = ModelWrapper(build_dir + "/yolo_dataflow_model.onnx")
fc_layers = model.get_nodes_by_op_type("MVAU_hls")

In [20]:
# each tuple is (PE, SIMD, in_fifo_depth) for a layer
folding = [
    (8, 9, [8]),
    (4, 4, [8]),
    (4, 4, [8]),
    (2, 2, [8]),
    (2, 2, [8]),
    (2, 2, [4]),
    (4, 4, [4]),
    (4, 4, [8]),
    (4, 4, [8]),
    (1, 8, [8]),
]

for fcl, (pe, simd, ififodepth) in zip(fc_layers, folding):
    fcl_inst = getCustomOp(fcl)
    fcl_inst.set_nodeattr("PE", pe)
    fcl_inst.set_nodeattr("SIMD", simd)
    fcl_inst.set_nodeattr("inFIFODepths", ififodepth)



In [21]:
fc_layers = model.get_nodes_by_op_type("ConvolutionInputGenerator_rtl")

In [22]:
folding = [
    (3),
    (2),
    (2),
    (2),
    (2),
    (2),
    (2),
    (2),
    (2),
]
for fcl, (simd) in zip(fc_layers, folding):
    fcl_inst = getCustomOp(fcl)
    fcl_inst.set_nodeattr("SIMD", simd)


In [23]:
model = model.transform(GiveUniqueNodeNames())
model.save(build_dir + "/yolo_folded.onnx")

In [24]:
showInNetron(build_dir + "/yolo_folded.onnx")

Stopping http://0.0.0.0:8081
Serving '/home/drmervat/Desktop/YOLO_GP/YOLO__/yolo_folded.onnx' at http://0.0.0.0:8081


In [25]:
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
model = ModelWrapper(build_dir+"/yolo_folded.onnx")
model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns))

/home/drmervat/Desktop/finn/src/finn/transformation/fpgadataflow/floorplan.py:107: UserWarning: 56 nodes have no entry in the provided floorplan, SLR was set to -1
  warnings.warn(
/home/drmervat/Desktop/finn/src/finn/transformation/fpgadataflow/insert_fifo.py:234: UserWarning: Input FIFO for IODMA_hls_0_out0 has depth 2 and won't
                        be created. This may cause RTL simulation issues.
                        
  warnings.warn(
/home/drmervat/Desktop/finn/src/finn/transformation/fpgadataflow/insert_fifo.py:294: UserWarning: Output FIFO for MVAU_hls_9_out0 has depth 2 and won't
                        be created. This may cause RTL simulation issues.
                        
  warnings.warn(
/home/drmervat/Desktop/finn/src/finn/transformation/fpgadataflow/create_stitched_ip.py:290: UserWarning: First node is not StreamingFIFO or IODMA.
                You may experience incorrect stitched-IP rtlsim or hardware
                behavior. It is strongly recommended to inse

In [26]:
from finn.transformation.fpgadataflow.make_pynq_driver import MakePYNQDriver
model = model.transform(MakePYNQDriver("zynq-iodma"))

In [27]:
model.save(build_dir + "/yolo_synth.onnx")
showInNetron(build_dir + "/yolo_synth.onnx")

Stopping http://0.0.0.0:8081
Serving '/home/drmervat/Desktop/YOLO_GP/YOLO__/yolo_synth.onnx' at http://0.0.0.0:8081


In [28]:
model = ModelWrapper(build_dir + "/yolo_synth.onnx")
model.model.metadata_props

[key: "floorplan_json"
value: "/tmp/finn_dev_drmervat/vitis_floorplan_sj2s58h5/floorplan.json"
, key: "vivado_pynq_proj"
value: "/tmp/finn_dev_drmervat/vivado_zynq_proj_kv__451b"
, key: "bitfile"
value: "/tmp/finn_dev_drmervat/vivado_zynq_proj_kv__451b/resizer.bit"
, key: "hw_handoff"
value: "/tmp/finn_dev_drmervat/vivado_zynq_proj_kv__451b/resizer.hwh"
, key: "vivado_synth_rpt"
value: "/tmp/finn_dev_drmervat/vivado_zynq_proj_kv__451b/synth_report.xml"
, key: "platform"
value: "zynq-iodma"
, key: "pynq_driver_dir"
value: "/tmp/finn_dev_drmervat/pynq_driver_u6qflmvr"
]

In [29]:
! ls {model.get_metadata_prop("vivado_pynq_proj")}

finn_zynq_link.cache	      finn_zynq_link.srcs  resizer.hwh
finn_zynq_link.gen	      finn_zynq_link.xpr   synth_project.sh
finn_zynq_link.hw	      ip_config.tcl	   synth_report.xml
finn_zynq_link.ip_user_files  NA		   vivado.jou
finn_zynq_link.runs	      resizer.bit	   vivado.log
